Thest chained T2 and T2 filter selection for SkyPortal submission.

In [ ]:
import os
%load_ext ampel_quick_import
%qi DevAmpelContext DictSecretProvider AmpelLogger T2Processor ChannelModel AlertConsumer TarAlertLoader ChannelModel AbsAlertFilter T3Processor TransientInfoPrinter

In [ ]:
# See other sample notebooks for path to download
ALERT_ARCHIVE = "/home/jnordin/data/ztfalerts/ztf_public_20180731_nostars.tar.gz"
AMPEL_CONF = "/home/jnordin/github/ampelv71/ampel_conf.yaml"

In [ ]:
secrets = DictSecretProvider( {'slack/ztf_general/jno':None,
                              'mongo/logger':{},
                              'mongo/writer':{},
                               'fritz/jvs/ampelbot': None,
                               'desi/skyportal/jno': None,
                              })

In [ ]:
ctx = DevAmpelContext.load(
    config_file_path = AMPEL_CONF,
    db_prefix = "DESITest",
    purge_db = True,
    secrets = secrets,
)

In [ ]:
# First channel, general look combined with SNGuess
ctx.add_channel(
    name="DESI_1",
    access=['ZTF', 'ZTF_PUB', 'ZTF_COLLAB']
)

In [ ]:
salt_conf = {
    "model": "salt2",
    "upper_limits": False,
    "plot_props": {
        "tags": ["SALT", "SNCOSMO"],
        "file_name": {
            "format_str": "%s_%s_fit.svg",
            "arg_keys": ["stock", "model"]
        },
        "title": {
            "format_str": "%s %s lightcurve fit",
            "arg_keys": ["stock", "model"]
        },
        "width": 10,
        "height": 6,
        "id_mapper": "ZTFIdMapper",
        "disk_save": "/home/jnordin/tmp/"
    }
}

In [ ]:
salt_config_id = ctx.add_config_id( salt_conf )
print(salt_config_id)

In [ ]:
ap = AlertConsumer(
    context = ctx,
    process_name = "ipyton_notebook_test",
    supplier = "ZiAlertSupplier",
    log_profile = "debug",
    directives = [
        {
            "channel": "DESI_1", 
            "filter": {"unit": "SimpleDecentFilter","config": {
#                                                                "min_drb":0.995,
                                                                "min_rb":0.3,
                                                                "min_ndet":3,
                                                                "min_tspan":0.02,
                                                                "max_tspan" : 999,
                                                                "min_gal_lat" : 14
                                                              }
                        },
            "stock_update": "ZiStockIngester",
            't0_add': {
                "ingester": "ZiAlertContentIngester",
                "t1_combine": [
                    {
                        "ingester": "PhotoCompoundIngester",
                        "config": {"combiner": "ZiT1Combiner"},
                        "t2_compute": {
                            "ingester": "PhotoT2Ingester",
                            "config": {"tags": ["ZTF"]},
                            "units": [
                                {'unit': 'T2SNCosmo',
                                 'config': salt_config_id
                                },
                                {'unit': 'T2BrightSNProb'
                                },
                            ]
                        }
                    }
                ],
            }
        }
    ]
)

In [ ]:
ap.set_loader(TarAlertLoader(ALERT_ARCHIVE))

In [ ]:
ap.set_iter_max(20)

In [ ]:
ap.run()

In [ ]:
t2p = T2Processor(context=ctx, process_name="T2Processor_test", log_profile="debug")

In [ ]:
t2p.run()

In [ ]:
t3p = T3Processor(
    context=ctx,
    process_name = "T3Processor_test",
    log_profile = "default", # debug
    channel = "DESI_1",
    directives = [ {
        "context": [
            {"unit": "T3AddLastRunTime"},
            {"unit": "T3AddAlertsNumber"}
        ],
        "select": {
            "unit": "T3FilteringStockSelector",
            "config": {
                "created": {"after": {"match_type": "time_delta", "days": -400000}},
                'tag': {'with': "ZTF", 'without': 'HAS_ERROR'},
                't2_filter':  {
                    'unit': 'T2SNCosmo',
                    'match': {'sncosmo_info.ndof':{"$gt":2} },
                }, 
            }
        },
        "load": {
            "unit": "T3SimpleDataLoader",
            "config": {
                "directives": ["TRANSIENT", "DATAPOINT", "COMPOUND", "T2RECORD"],
            }

        },
        "run": {
            "unit": "T3UnitRunner",
            "config": {
                "directives": [
                      {

                            "project": {
                                "unit": "T3ChannelProjector",
                                "config": {
                                    "channel": "DESI_1"
                                }
                            },
                            "execute": [
                                {
                                    "unit": "RapidBase",
                                    "config": {
                                        'do_react': False,
                                        'do_testreact': True,
                                        'slack_token': {
                                            "key": "slack/ztf_general/jno",
                                        },
                                        'slack_channel': '#ztf_auto',
                                        'slack_username': 'AMPEL',
                                        't2info_from' : []
                                    },
                                },
                                {
                                    "unit": "SkyPortalPublisher",
                                    "config": {
                                        'token': {
                                            "key": "fritz/jvs/ampelbot",
                                        },
                                        'base_url': 'https://fritz.science',
                                        'filters': ['AMPEL.OKC_LENSED'],
                                    },
                                },
                            ]

                      }
                ]
            }
        }
    } ]
)


In [ ]:
t3p.run()